<A HREF="https://huggingface.co/blog/sentiment-analysis-python">Getting started with sentiment analysis using huggingface</A>

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import en_core_web_md
import csv
import regex as re
import os
import spacy
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim import corpora
import pprint
from transformers import pipeline # huggingface

In [37]:
# Setup huggingface sentiment pipeline - default, distilbert, is 255MB
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [38]:
# Load up the files
#path = './Data/DataUCSB/' # Smaller UCSB dataset
path = './Data/amrhet/' # larger American Rhetoric dataset
list_of_files = []

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.txt'):
            list_of_files.append(os.path.join(root,file))

filepath = './Data/DataUCSB/address-before-joint-session-the-congress-the-state-the-union-16.csv'
speeches = []
for file in list_of_files:
    with open(file, encoding='utf-8') as f:
        text = f.read()
    f.close()
    speeches.append(text)

#clean out xa0 space characters
[speech.replace(u'\xa0', '') for speech in speeches if len(speech)>0]; # ; hides output of list comprehension

In [39]:
test = speeches[343].split('.') # sentiment_pipeline wants sentences! Not whole speeches.
sent = sentiment_pipeline(test)

In [40]:
round(sum( [sent[i]['score'] for i in range(len(sent))] )/len(sent), 3)

0.959

In [41]:
# Stuff for progress bar
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations

In [42]:
# This is slow for 466 speeches!
scores = []
# Wrap iterable in tqdm() for progress bar
for i in tqdm(range(len(speeches))):
    test = speeches[i].split('.')
    sent = sentiment_pipeline(test)
    scores.append(sum( [sent[i]['score'] for i in range(len(sent))] )/len(sent))

  0%|          | 0/466 [00:00<?, ?it/s]

In [43]:
np.min(scores)

0.748120903968811

In [44]:
np.max(scores)

0.992634262641271

In [45]:
np.mean(scores)

0.9604947458079558

In [46]:
np.var(scores)

0.00048181062130331153